# Flood Model Training Notebook

Train a Flood ConvLSTM Model using `usl_models` lib.

In [12]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from usl_models.flood_ml import constants
from usl_models.flood_ml.model import FloodModel, FloodModelParams
from usl_models.flood_ml.dataset import load_dataset_windowed, load_dataset
sim_names=["Manhattan-config_v1/Rainfall_Data_1.txt"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
model = FloodModel(FloodModelParams())
dataset = load_dataset_windowed(sim_names=sim_names, batch_size=4, max_chunks=8)
model.fit(dataset, epochs=1)

38/38 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - loss: 0.0063 - mean_absolute_error: 0.0053 - root_mean_squared_error: 0.0655


2024-07-16 19:35:10.853659: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-16 19:35:10.853722: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2024-07-16 19:35:10.853752: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4627997881443418559
2024-07-16 19:35:10.853764: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14211259310590093937
2024-07-16 19:35:10.853775: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 132374840450864031
2024-07-16 19:35:10.853805: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13240268832531781109
/opt/conda

In [9]:
# Test calling the model on some data.
inputs, labels_ = next(iter(dataset))
prediction = model.call(inputs)
prediction.shape

TensorShape([4, 1000, 1000, 1])

In [13]:
# Test calling the model for n predictions
full_dataset = load_dataset(sim_names=sim_names, batch_size=1)
inputs, labels_ = next(iter(full_dataset))
predictions = model.call_n(inputs, n=4)
predictions.shape

TensorShape([1, 4, 1000, 1000])